# PaddleOCR on SageMaker--推理

## 0 运行环境
本文在boto3 1.15.16和sagemaker 2.15.0下测试通过。

In [ ]:
import boto3,sagemaker
print(boto3.__version__)
print(sagemaker.__version__)

可运行下行代码进行升级，升级完毕后，需要重启Kernal，然后再检查版本

In [ ]:
!pip install sagemaker --upgrade -i https://opentuna.cn/pypi/web/simple/

## 1 准备image
分为2种方式，分别为获取现有image，以及自行Build image

### 1.1 获取现有image

本项目已build完毕image，存放到ECR中，可直接部署到SageMaker。选择合适版本后，可直接跳到 **2部署到SageMaker** 章节  
**tag**中的**2.0.0rc0**指PaddlePaddle版本，**1.1**指PaddleOCR模型版本

#### 1.1.1 使用CPU版本

In [ ]:
image_uri="048912060910.dkr.ecr.cn-northwest-1.amazonaws.com.cn/nwcd/ocr-inference:2.0.0rc0-1.1-cpu"

#### 1.1.2 使用GPU版本

In [ ]:
image_uri="048912060910.dkr.ecr.cn-northwest-1.amazonaws.com.cn/nwcd/ocr-inference:2.0.0rc0-1.1-gpu"

### 1.2 自行Build image
#### 1.2.1 准备PaddleOCR

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleOCR dockersource

#### 1.2.2 拷贝web相关文件到docker build目录

In [ ]:
!cp -r source/* dockersource

#### 1.2.3 准备模型
如果有自己的模型，可以直接存放到`dockersource/inference`目录下，如果没有，可直接使用官方的模型

In [ ]:
%cd dockersource
!mkdir inference
%cd inference
# 下载超轻量级中文OCR模型的检测模型并解压
!wget https://paddleocr.bj.bcebos.com/20-09-22/mobile/det/ch_ppocr_mobile_v1.1_det_infer.tar && tar xf ch_ppocr_mobile_v1.1_det_infer.tar && rm ch_ppocr_mobile_v1.1_det_infer.tar
# 下载超轻量级中文OCR模型的识别模型并解压
!wget https://paddleocr.bj.bcebos.com/20-09-22/mobile/rec/ch_ppocr_mobile_v1.1_rec_infer.tar && tar xf ch_ppocr_mobile_v1.1_rec_infer.tar && rm ch_ppocr_mobile_v1.1_rec_infer.tar
# 下载超轻量级中文OCR模型的文本方向分类器模型并解压
!wget https://paddleocr.bj.bcebos.com/20-09-22/cls/ch_ppocr_mobile_v1.1_cls_infer.tar && tar xf ch_ppocr_mobile_v1.1_cls_infer.tar && rm ch_ppocr_mobile_v1.1_cls_infer.tar
%cd ../..

#### 1.2.4 设置相关名称

In [ ]:
import boto3
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')
ecr_repository = 'ocr-inference'
tag = '2.0.0rc0-1.1-cpu'
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + ":" + tag)
print(image_uri)
ecr = '{}.dkr.ecr.{}.{}'.format(account_id, region, uri_suffix)

#### 1.2.5 Build image
CPU推理使用Dockerfile，GPU推理使用Dockerfile.gpu

In [ ]:
%%time
!docker build -t $ecr_repository:$tag -f Dockerfile .

#### 1.2.6 本地推理(可选)

CPU推理

In [ ]:
!docker run -p 8080:8080 -d --rm $ecr_repository:$tag serve

GPU推理

In [ ]:
!nvidia-docker run -p 8080:8080 -d --rm $ecr_repository:$tag serve

发送请求

In [ ]:
!curl -H "Content-Type: application/json" -X POST --data '{"bucket":"nowfox","image_uri":["data/test/12.jpg"]}' http://127.0.0.1:8080/invocations

#### 1.2.7 把image push到ECR

In [ ]:
!aws ecr create-repository --repository-name $ecr_repository

In [ ]:
!docker tag $ecr_repository:$tag $image_uri
!$(aws ecr get-login --no-include-email)
!docker push $image_uri

## 2 部署到SageMaker
CPU推理目前只能使用large机型，其他机型会报`Intel MKL function load error: cpu specific dynamic library is not loaded.`

In [ ]:
%%time
from sagemaker.model import Model
import boto3

iam = boto3.client('iam')
roles = iam.list_roles(PathPrefix='/service-role')
role=""
for current_role in roles["Roles"]:
    if current_role["RoleName"].startswith("AmazonSageMaker-ExecutionRole-"):
        role=current_role["Arn"]
        break
#如果role为空表示有问题
print(role)

endpoint_name = "ocr"
my_model = Model(role=role,image_uri=image_uri)

#该步骤,CPU版本大概需要7-10分钟，GPU版本大概需要10-15分钟
xgb_predictor = my_model.deploy(initial_instance_count=1,
                                endpoint_name=endpoint_name,
                                instance_type='ml.m5.large'
                                )

## 3 推理

### 准备测试图片

In [ ]:
data = {"bucket":"nowfox","image_uri":["data/test/12.jpg"]}

返回的结果为textract格式

### 方式一、使用boto3

In [ ]:
import boto3
import json
endpoint_name = "ocr"
client = boto3.client('runtime.sagemaker')

response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  Body=json.dumps(data),
                                  ContentType="application/json")
#print(response)
response_body = response['Body'] 
body= response_body.read()
result = json.loads(body.decode('utf-8'))
print(result)

### 方式二、使用SageMaker SDK

In [ ]:
import sagemaker
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer())
result = predictor.predict(data)
print(result)

## 4 删除Endpoint

In [ ]:
import boto3
client = boto3.client(service_name='sagemaker') 
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_name)